In [2]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/
!ls /home/kesci/input/gzlt/train_set/
!ls /home/kesci/input/gzlt/train_set/201708n/
!ls /home/kesci/input/gzlt/train_set/201708q/
!ls /home/kesci/input/gzlt/test_set/
!ls /home/kesci/input/gzlt/test_set/201808/

gzlt
201708n  201708q  weather_data_2017
201708n1.txt  201708n3.txt  201708n5.txt  201708n7.txt
201708n2.txt  201708n4.txt  201708n6.txt
201708q1.txt  201708q3.txt  201708q5.txt  201708q7.txt
201708q2.txt  201708q4.txt  201708q6.txt
201808	weather_data_2018
2018_1.txt  2018_3.txt	2018_5.txt  2018_7.txt
2018_2.txt  2018_4.txt	2018_6.txt


In [3]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

kesci_submit  lost+found  mysubmission.csv


In [4]:
# 查看当前kernerl下的package
#!pip list --format=columns

In [5]:
# 显示cell运行时长
%load_ext klab-autotime

In [6]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics

time: 1.08 s


In [7]:
def r_d(f):
    n1= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s1.txt"%(f,f), sep='\t',header=None)
    n1.columns=['账期','id','出账收入','labe']
    n2= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s2.txt"%(f,f), sep='\t',header=None)
    #print(n2.shape,'************\n',n2[:5])
    n2.columns=['id','手机品牌','终端型号','首次使用时间','末次使用时间','labe']
    n3= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s3.txt"%(f,f), sep='\t',header=None)
    #print(n3.shape,'************\n',n3[:5])
    n3.columns=['账期','id','联络圈规模','是否出省','是否出境','labe']
    n4= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s4.txt"%(f,f), sep='\t',header=None)
    #print(n4.shape,'************\n',n4[:5])
    n4.columns=['账期','id','漫出省份','labe']
    n5= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s5.txt"%(f,f), sep='\t',header=None)
    #print(n5.shape,'************\n',n5[:5])
    n5.columns=['账期','id','是否去过黔东南目标景区','labe']
    n6= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s6.txt"%(f,f), sep='\t',header=None)
    #print(n6.shape,'************\n',n6[:5])
    n6.columns=['日期','时段','id','经度','纬度','labe']
    n7= pd.read_csv("/home/kesci/input/gzlt/train_set/201708%s/201708%s7.txt"%(f,f), sep='\t',header=None)
    #print(n7.shape,'************\n',n7[:5])
    n7.columns=['账期','id','APP名称','流量','labe']
    uid=pd.DataFrame(list(set(list(n1['id'])+list(n2['id'])+list(n3['id'])+list(n4['id'])+\
        list(n5['id'])+list(n6['id'])+list(n7['id']))),columns=['id'])
    return n1,n2,n3,n4,n5,n6,n7,uid

time: 3.3 ms


In [8]:
test1= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_1.txt", sep='\t',header=None)
#print(test1.shape,'************\n',test1[:5])
test1.columns=['账期','id','出账收入']
test2= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_2.txt", sep='\t',header=None)
#print(test2.shape,'************\n',test2[:5])
test2.columns=['id','手机品牌','终端型号','首次使用时间','末次使用时间']
test3= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_3.txt", sep='\t',header=None)
#print(test3.shape,'************\n',test3[:5])
test3.columns=['账期','id','联络圈规模','是否出省','是否出境']
test4= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_4.txt", sep='\t',header=None)
#print(test4.shape,'************\n',test4[:5])
test4.columns=['账期','id','漫出省份']
test5= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_5.txt", sep='\t',header=None)
#print(test5.shape,'************\n',test5[:5])
test5.columns=['账期','id','是否去过黔东南目标景区']
test6= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_6.txt", sep='\t',header=None)
#print(test6.shape,'************\n',test6[:5])
test6.columns=['日期','时段','id','经度','纬度']
test7= pd.read_csv("/home/kesci/input/gzlt/test_set/201808/2018_7.txt", sep='\t',header=None)
#print(test7.shape,'************\n',test7[:5])
test7.columns=['账期','id','APP名称','流量']
test_uid=pd.DataFrame(list(set(list(test1['id'])+list(test2['id'])+list(test3['id'])+list(test4['id'])+\
        list(test5['id'])+list(test6['id'])+list(test7['id']))),columns=['id'])
print(test_uid.shape)

(50200, 1)
time: 19.5 s


In [9]:
def f1(d):
    '''每月的出账收入'''
    d=d.groupby(['id','账期'],as_index=False)['出账收入'].agg({'出账收入':'mean'})
    riqi=list(set(d['账期']))
    nd=pd.DataFrame(list(set(d['id'])),columns=['id'])
    for i in riqi:
        m=d[d['账期']==i][['id','出账收入']]
        m.columns=['id',str(i)+'出账收入']
        nd=pd.merge(nd,m,how='left',on=['id'])
    del riqi,m,d
    return nd

time: 1.64 ms


In [10]:
n1,n2,n3,n4,n5,n6,n7,n_uid=r_d(f='n')
q1,q2,q3,q4,q5,q6,q7,q_uid=r_d('q')
print(len(n_uid))

139903
time: 41.4 s


In [12]:
import time
def f2(d):
    d=d.fillna(-99)
    #nd=pd.DataFrame(list(set(d['id'])),columns=['id'])
    m=d.groupby(['id'],as_index=False)['id'].agg({'id_count_2':'count'})
    m1=d[d['手机品牌']==-99].groupby(['id'],as_index=False)['手机品牌'].agg({'手机品牌_2':'count'})
    m=pd.merge(m,m1,how='left',on=['id']).fillna(0)
    m['手机品牌缺失率_2']=m['手机品牌_2']/m['id_count_2']
    
    m1=d[d['手机品牌']==-99].groupby(['id'],as_index=False)['手机品牌'].agg({'手机品牌nunique_2':'nunique'})
    m=pd.merge(m,m1,how='left',on=['id'])
    
    m1=d[d['终端型号']==-99].groupby(['id'],as_index=False)['终端型号'].agg({'终端型号nunique_2':'nunique'})
    m=pd.merge(m,m1,how='left',on=['id'])
    
    d['末次使用时间']=d['末次使用时间'].apply(lambda x:\
    int(time.mktime(time.strptime(str(int(x)), "%Y%m%d%H%M%S"))) if int(x)>0 else -99)
    d['首次使用时间']=d['首次使用时间'].apply(lambda x:\
    int(time.mktime(time.strptime(str(int(x)), "%Y%m%d%H%M%S"))) if int(x)>0 else -99)
    d['末-首次使用时间']=(d['末次使用时间']-d['首次使用时间'])/3600
    
    m1=d.groupby(['id'],as_index=False)['末-首次使用时间'].agg({'末-首次使用时间mean':'mean',\
        '末-首次使用时间max':'max',\
        '末-首次使用时间min':'min',\
        '末-首次使用时间std':'std',\
         })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    ##加小时，周期特征：
    return m

(186800, 6) 
******************
 账期           2
id       93400
联络圈规模      533
是否出省         2
是否出境         2
labe         1
dtype: int64
           账期                id  联络圈规模  是否出省  是否出境  labe
11199  201706  1030042537821359     83     0     0     1
5599   201707  1030042537821359     78     0     0     1
5598   201707  1030042459517639     74     0     0     1
11198  201706  1030042459517639     66     0     0     1
5597   201707  1030042459513741     79     0     0     1
11197  201706  1030042459513741     71     0     0     1
5596   201707  1030042459510223     69     1     0     1
11196  201706  1030042459510223     86     1     0     1
11195  201706  1030042459508623      8     0     0     1
5595   201707  1030042459508623     15     0     0     1
time: 27 ms


In [13]:
def f3(d):
    m=pd.DataFrame(list(set(d['id'])),columns=['id'])
    m1=d[d['联络圈规模']!=-1].groupby(['id'],as_index=False)['联络圈规模'].agg({'联络圈规模_3':'mean'})
    m=pd.merge(m,m1,how='left',on=['id']).fillna(-1)
    
    m1=d.groupby(['id','账期'],as_index=False)['是否出省'].agg({'是否出省':'mean'})
    for i in list(set(m1['账期'])):
        mm=m1[m1['账期']==i][['id','是否出省']]
        mm.columns=['id',str(i)+'是否出省']
        m=pd.merge(m,mm,how='left',on=['id'])
    m1=d.groupby(['id','账期'],as_index=False)['是否出境'].agg({'是否出境':'mean'})
    for i in list(set(m1['账期'])):
        mm=m1[m1['账期']==i][['id','是否出境']]
        mm.columns=['id',str(i)+'是否出境']
        m=pd.merge(m,mm,how='left',on=['id'])
    del m1
    return m

time: 2.77 ms


In [14]:
def f4(d):
    m=d.groupby(['id'],as_index=False)['漫出省份'].agg({'漫出省份count_4':'count',\
       '漫出省份nunique_4':'nunique'
    })
    m1=d[d['漫出省份'].isnull()].groupby(['id'],as_index=False)['漫出省份'].agg({'漫出省份缺失_4':'count'})
    m=pd.merge(m,m1,how='left',on=['id']).fillna(0)
    m['漫出省份缺失率']=m['漫出省份缺失_4']/m['漫出省份count_4']
    
    d=d.fillna(-1)
    zz=[]
    for i in m['id']:
        z={}
        m1=d[d['id']==i]['漫出省份']
        for j in m1:
            z[j]=z.get(j,0)+1
        zz.append(z)
    zz=pd.DataFrame(zz)
    m=pd.concat([m,zz],axis=1)
    del m1,z,zz,
    return m

(36499, 4) 
******************
 账期          2
id      12958
漫出省份       31
labe        1
dtype: int64
          账期                id 漫出省份  labe
7288  201706  1030042459510223   广东     1
7283  201706  1030042459510223   广东     1
3925  201707  1030042459510223   上海     1
3927  201707  1030042459510223   浙江     1
7282  201706  1030042459510223   山东     1
3926  201707  1030042459510223   江苏     1
7284  201706  1030042459510223   广东     1
7285  201706  1030042459510223   山东     1
7287  201706  1030042459510223   广东     1
7286  201706  1030042459510223   广东     1
[nan, '江苏', '河南', '甘肃', '天津', '上海', '山西', '江西', '浙江', '辽宁', '青海', '广西', '云南', '河北', '陕西', '安徽', '湖北', '贵州', '宁夏', '重庆', '四川', '北京', '广东', '吉林', '山东', '福建', '新疆', '海南', '黑龙江', '湖南', '西藏', '内蒙古']
(400, 4)
time: 29.1 ms


In [15]:
def f5(d):
    m=pd.DataFrame(list(set(d['id'])),columns=['id'])
    for i in list(set(d['账期'])):
        m1=d[d['账期']==i][['id','是否去过黔东南目标景区']]
        m1.columns=['id',str(i)+'是否去过黔东南目标景区']
        m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    return m


(11200, 4) 
******************
 账期                2
id             5600
是否去过黔东南目标景区       2
labe              1
dtype: int64
time: 5.68 ms


In [16]:
def f6(d):
    d['基站']=d['经度'].astype(str)+'_'+d['纬度'].astype(str)
    m=d.groupby(['id'],as_index=False)['基站'].agg({'基站count_6':'count',\
       '基站nunique_6':'nunique'
    })
    return m 

                 日期    时段              id          经度        纬度  labe
1836418  2017-06-01  18.0  10000000050393  106.712090  26.58831     1
2230035  2017-06-01   2.0  10000000050393  106.712090  26.58831     1
2526340  2017-06-01  14.0  10000000050393  106.724756  26.56594     1
2756403  2017-06-01  20.0  10000000050393  106.709940  26.58616     1
2822199  2017-06-01  16.0  10000000050393  106.712090  26.58831     1
1705047  2017-06-02  16.0  10000000050393  106.724756  26.56594     1
2032962  2017-06-02  14.0  10000000050393  106.724756  26.56594     1
2065723  2017-06-02   5.0  10000000050393  106.712090  26.58831     1
2131401  2017-06-02  22.0  10000000050393  106.713790  26.58892     1
2263032  2017-06-02  12.0  10000000050393  106.724756  26.56594     1
(2852871, 6) 
******************
 日期         62
时段         13
id       5575
经度      23355
纬度      23226
labe        1
dtype: int64
time: 3.19 s


In [17]:
def f7(d):
    m=d.groupby(['id'],as_index=False)['APP名称'].agg({'APP名称count_7':'count',\
       'APP名称nunique_7':'nunique'
    })
    m1=d.groupby(['id'],as_index=False)['流量'].agg({'流量mean_7':'mean',\
       '流量sum_7':'sum',\
       '流量max_7':'max',\
       '流量min_7':'min',\
       '流量std_7':'std',\
    })
    m=pd.merge(m,m1,how='left',on=['id'])
    del m1
    return m

(173117, 5) 
******************
 账期           2
id        4717
APP名称      762
流量       11485
labe         1
dtype: int64
            账期              id      APP名称     流量  labe
11537   201707  10000000050393       腾讯新闻   0.00     1
133735  201706  10000000050393        沃音乐   0.01     1
161850  201706  10000000050393       腾讯视频   0.01     1
65921   201707  10000000050393       高德地图   0.01     1
17601   201707  10000000050393     华为应用市场   0.13     1
145197  201706  10000000050393         微信  11.63     1
929     201707  10000000050393       小米商城   0.01     1
27760   201707  10000000050393         QQ   0.64     1
94381   201706  10000000050393       小米商城   0.01     1
60285   201707  10000000050393        唯品会   2.70     1
121992  201706  10000000050393         QQ   0.19     1
147207  201706  10000000050393       携程旅行   1.49     1
106817  201706  10000000050393        支付宝   0.02     1
65069   201707  10000000050393      爱奇艺视频   0.00     1
25769   201707  10000000050393       神州专车   0.20     1

In [18]:
q_uid['label']=1
n_uid['label']=0
q_train=pd.merge(q_uid,f1(q1),how='left',on=['id'])
q_train=pd.merge(q_train,f2(q2),how='left',on=['id'])
q_train=pd.merge(q_train,f3(q3),how='left',on=['id'])
q_train=pd.merge(q_train,f4(q4),how='left',on=['id'])
q_train=pd.merge(q_train,f5(q5),how='left',on=['id'])
q_train=pd.merge(q_train,f6(q6),how='left',on=['id'])
q_train=pd.merge(q_train,f7(q7),how='left',on=['id'])

n_train=pd.merge(n_uid,f1(n1),how='left',on=['id'])
n_train=pd.merge(n_train,f2(n2),how='left',on=['id'])
n_train=pd.merge(n_train,f3(n3),how='left',on=['id'])
n_train=pd.merge(n_train,f4(n4),how='left',on=['id'])
n_train=pd.merge(n_train,f5(n5),how='left',on=['id'])
n_train=pd.merge(n_train,f6(n6),how='left',on=['id'])
n_train=pd.merge(n_train,f7(n7),how='left',on=['id'])

test=pd.merge(test_uid,f1(test1),how='left',on=['id'])
test=pd.merge(test,f2(test2),how='left',on=['id'])
test=pd.merge(test,f3(test3),how='left',on=['id'])
test=pd.merge(test,f4(test4),how='left',on=['id'])
test=pd.merge(test,f5(test5),how='left',on=['id'])
test=pd.merge(test,f6(test6),how='left',on=['id'])
test=pd.merge(test,f7(test7),how='left',on=['id'])

print(q_train.shape,n_train.shape,test.shape)

(5600, 64) (139903, 65) (50200, 64)
time: 7min 10s


In [25]:
train=pd.concat([q_train,n_train],axis=0).reset_index(drop=True)
label=train['label']


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


0    139903
1      5600
Name: label, dtype: int64
6015600067 1030042537821359
99000
10000000005691
10000000050393 1036145391789885
相同用户个数： 3519
3519
相同用户标签分布： 1    3262
0     257
Name: label, dtype: int64
time: 4.67 s


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
def xgb_model(new_train,y,new_test):
  '''定义模型'''
  xgb_params = {'booster': 'gbtree',
          'eta': 0.01, 'max_depth': 5, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective':'binary:logistic',
          'eval_metric': 'auc',
          'silent': True,
          }
  #skf=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=2018)
  skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
  oof_xgb=np.zeros(len(new_train))
  prediction_xgb=np.zeros(len(new_test))
  for i,(tr,va) in enumerate(skf.split(new_train,y)):
    print('fold:',i+1,'training')
    dtrain = xgb.DMatrix(new_train[tr],y[tr])
    dvalid = xgb.DMatrix(new_train[va],y[va])
    watchlist = [(dtrain, 'train'), (dvalid, 'valid_data')]
    bst = xgb.train(dtrain=dtrain, num_boost_round=30000, evals=watchlist, early_stopping_rounds=200, verbose_eval=200, params=xgb_params)
    oof_xgb[va] += bst.predict(xgb.DMatrix(new_train[va]), ntree_limit=bst.best_ntree_limit)
    prediction_xgb += bst.predict(xgb.DMatrix(new_test), ntree_limit=bst.best_ntree_limit)
  print('the roc_auc_score for train:',roc_auc_score(y,oof_xgb))
  prediction_xgb/=5
  return oof_xgb,prediction_xgb
def lgb_model(new_train,y,new_test):
    params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_leaves': 1000,
    'verbose': -1,
    'max_depth': -1,
  #  'reg_alpha':2.2,
  #  'reg_lambda':1.4,
    'seed':42,
    }
    #skf=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=2018)
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    oof_lgb=np.zeros(len(new_train))
    prediction_lgb=np.zeros(len(new_test))
    feature_importance_df = pd.DataFrame()
    for i,(tr,va) in enumerate(skf.split(new_train,y)):
        print('fold:',i+1,'training')
        dtrain = lgb.Dataset(new_train.loc[tr],y[tr])
        dvalid = lgb.Dataset(new_train.loc[va],y[va],reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=30000, valid_sets=dvalid, verbose_eval=400,early_stopping_rounds=200)
        oof_lgb[va] += bst.predict(new_train.loc[va], num_iteration=bst.best_iteration)
        prediction_lgb += bst.predict(new_test, num_iteration=bst.best_iteration)
    
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = list(new_train.columns)
        fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
    print('the roc_auc_score for train:',roc_auc_score(y,oof_lgb))
    prediction_lgb/=5
    return oof_lgb,prediction_lgb,feature_importance_df

time: 103 ms


In [21]:
from sklearn.utils import shuffle
oof_lgb,prediction_lgb,feature_importance_df=\
lgb_model(train.drop(['id','label'],axis=1),label,test.drop(['id'],axis=1))

fold: 1 training
Training until validation scores don't improve for 200 rounds.
[400]	valid_0's auc: 0.988638
[800]	valid_0's auc: 0.989182
Early stopping, best iteration is:
[871]	valid_0's auc: 0.989271
fold: 2 training
Training until validation scores don't improve for 200 rounds.
[400]	valid_0's auc: 0.985631
[800]	valid_0's auc: 0.9863
Early stopping, best iteration is:
[673]	valid_0's auc: 0.986347
fold: 3 training
Training until validation scores don't improve for 200 rounds.
[400]	valid_0's auc: 0.987702
[800]	valid_0's auc: 0.988018
Early stopping, best iteration is:
[910]	valid_0's auc: 0.988059
fold: 4 training
Training until validation scores don't improve for 200 rounds.
[400]	valid_0's auc: 0.987967
[800]	valid_0's auc: 0.988441
[1200]	valid_0's auc: 0.988492
Early stopping, best iteration is:
[1070]	valid_0's auc: 0.988671
fold: 5 training
Training until validation scores don't improve for 200 rounds.
[400]	valid_0's auc: 0.989911
[800]	valid_0's auc: 0.990455
Early stop

In [22]:
oof_xgb,prediction_xgb=\
xgb_model(np.array(train.drop(['id','label'],axis=1)),label,np.array(test.drop(['id'],axis=1)))

fold: 1 training
[0]	train-auc:0.976254	valid_data-auc:0.972295


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train until valid_data-auc hasn't improved in 200 rounds.
[200]	train-auc:0.989506	valid_data-auc:0.988515
[400]	train-auc:0.990533	valid_data-auc:0.988021
Stopping. Best iteration:
[209]	train-auc:0.989797	valid_data-auc:0.988845

fold: 2 training
[0]	train-auc:0.974975	valid_data-auc:0.968626
Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train until valid_data-auc hasn't improved in 200 rounds.
[200]	train-auc:0.990046	valid_data-auc:0.98396
[400]	train-auc:0.991233	valid_data-auc:0.985781
[600]	train-auc:0.992736	valid_data-auc:0.987824
[800]	train-auc:0.994069	valid_data-auc:0.988232
[1000]	train-auc:0.99549	valid_data-auc:0.988311
Stopping. Best iteration:
[997]	train-auc:0.995471	valid_data-auc:0.988316

fold: 3 training
[0]	train-auc:0.974694	valid_data-auc:0.971352
Multiple eval metrics have been passed: 'valid_data-auc' will be used 

In [23]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
r=[1 if i>0.4 else 0 for i in oof_xgb]
print('confusion_matrix for lr:\n', metrics.confusion_matrix(label,r))
print('classification_report for lr\n:', metrics.classification_report(label,r))

confusion_matrix for lr:
 [[139880     23]
 [   605   4995]]
classification_report for lr
:               precision    recall  f1-score   support

           0       1.00      1.00      1.00    139903
           1       1.00      0.89      0.94      5600

    accuracy                           1.00    145503
   macro avg       1.00      0.95      0.97    145503
weighted avg       1.00      1.00      1.00    145503

time: 378 ms


In [40]:
sub=test[['id']].astype(str)
sub['Pred']=prediction_xgb
'''
for i in range(len(xiu2)):
    if int(xiu2.loc[i,'label'])==1:
        sub.loc[sub.id==str(xiu2.loc[i,'id']),'Pred']=0#xiu2.loc[i,'label']
    else:
        sub.loc[sub.id==str(xiu2.loc[i,'id']),'Pred']=1
'''
sub=sub[~sub.id.isin([str(i) for i in list(xiu2.id)])].reset_index(drop=True)

sub['Pred']=sub['Pred'].apply(lambda x:round(x,6))
sub.columns=['ID','Pred']
sub.to_csv('mysubmission.csv',index=None)
print(sub['Pred'].describe())
print(len(sub[sub.Pred>0.6]))

count    46681.000000
mean         0.479861
std          0.251511
min          0.015098
25%          0.381349
50%          0.579835
75%          0.593643
max          0.957090
Name: Pred, dtype: float64
11271
time: 203 ms


In [89]:
#!wget -nv -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
2019-06-18 06:19:59 URL:https://www.heywhale.com/kesci_submit [7842088/7842088] -> "kesci_submit" [1]
time: 1.82 s


Kesci Submit Tool
Result File: mysubmission.csv (1.15 MiB)
Uploaded.       
Submit Success.
{"Stage":0,"Status":0,"ShownInHistory":true,"IsAucResult":true,"Selected":false,"_id":"5d09ab5e921a91002b8dd863","Competition":"5be92233954d6e001063649a","Team":"5d0476d0902bad002c295284","UploadDate":"2019-06-19T03:26:22.404Z","Final":true,"Response":"","SubmissionResults":[],"IP":"52.82.22.223","FingerPrint":"","UserAgent":"Go-http-client/1.1","ResultFileName":"1560914781999c4a7f9.csv","ResultFileRealName":"mysubmission.csv","ResultFileSize":0,"ReviewInfos":[],"__v":0}

time: 1.54 s
